# NSU, vaje 3

### A: Meta klasifikacija in meta regresija.
A.1: Uporabi meta podatke iz prejsnjih dveh nalog. Razbij jih na testno in ucno mnozico. 
pyMFE nekaterih meta značilk ni mogel izračunati, zato podatki vsebujejo nekaj manjkajočih vrednosti. Odstrani stolpce z njimi.

In [1]:
import numpy as np
import pandas as pd

x_meta_all = pd.read_csv("meta_features.csv")
x_meta = x_meta_all.dropna(axis=1).drop("name", axis=1)
y_meta_all = pd.read_csv("meta_target.csv")
print(y_meta_all.head())

            name       knn      tree     bayes   best
0           iris  0.973684  0.947368  0.973684    knn
1            zoo  0.807692  0.923077  0.923077   tree
2           wine  0.644444  0.888889  0.977778  bayes
3     hayes-roth  0.550000  0.900000  0.725000   tree
4  fri_c3_100_50  0.520000  0.880000  0.560000   tree


A.2: Natreniraj meta model z metodo naključnega gozda, ki bo napovedal, katera izmed metod drevo, 1NN in Bayes dela najbolje na sestavljenih meta podatkih. 
Za testiranje uporabi 5-kratno prečno preverjanje. Natančnost meta modela primerjaj z natančnostjo modela, ki vedno napove povprečno vrednost ciljne spremenjivke na učni množici (pomagaš si lahko z **sklearn.dummy.DummyClassifier**).

Kakšna se ti zdi uspešnost meta modela?

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier


Y = y_meta_all['best']

rf = RandomForestClassifier()
cvrf = cross_validate(rf, x_meta, Y, cv=5,
    scoring=(['accuracy']),
    return_train_score=True)

print(np.mean(cvrf['test_accuracy']))

dummy = DummyClassifier(strategy="most_frequent")
cvdummy = cross_validate(dummy, x_meta, Y, cv=5,
    scoring=(['accuracy']),
    return_train_score=True)

print(np.mean(cvdummy['test_accuracy']))

0.5781818181818182
0.4618181818181818


A.3: Naše meta podatke sestavlja kar nekaj različnih meta značilk. Katere so pa zares pomembne in koristne? 
Nekatere metode strojnega učenja nam omogočajo oceno pomembnosti značilk, ki nam lahko pomaga odgovoriti na to vprašanje. Primeri takšnih metod so logistična regresija (izračunamo iz koeficientov **model.coef_**), odločitveno drevo (**model.feature_importances_**) in naključni gozd (**model.feature_importances_**).

Katerih 5 meta značilk je najpomembnejših? Izriši še stolpični diagram (**plt.bar**) pomembnosti meta značilk.

In [47]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

#X_train, X_test, y_train, y_test = train_test_split(x_meta, Y, test_size=0.2, random_state=42)
lr = LogisticRegression().fit(x_meta, Y)
implr = lr.coef_
#print(np.argsort(implr)[0][-6:-1])
print(x_meta.columns[np.argsort(implr)[0][-6:-1]])


rfo = RandomForestClassifier().fit(x_meta, Y)
imprf = rfo.feature_importances_
#print(imprf)

print(x_meta.columns[np.argsort(imprf)[-6:-1]])

tree = DecisionTreeClassifier().fit(x_meta, Y)
imptree = tree.feature_importances_
#print(imptree)
print(x_meta.columns[np.argsort(imptree)[-6:-1]])


# v resitvah se izris stolpicnega diagrama

Index(['leaves_homo.sd', 'tree_depth.mean', 'nr_inst', 'tree_depth.sd',
       'leaves_branch.sd'],
      dtype='object')
Index(['attr_conc.mean', 'class_conc.mean', 'attr_conc.sd', 'attr_ent.sd',
       'tree_shape.sd'],
      dtype='object')
Index(['var_importance.sd', 'leaves_branch.sd', 'nodes_per_level.mean',
       'leaves_homo.sd', 'tree_shape.sd'],
      dtype='object')


/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


A.4: Preizkusimo še meta regresijo. Za vsako od naših treh metod strojnega učenja nauči meta model, ki napoveduje njegovo natančnost. Kolikšen je $R^2$ vsakega meta modela, izračunana s 5-kratnim prečnim preverjanjem?

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor

metode = {"knn", "tree", "bayes"}
for metoda in metode:
    y_meta = y_meta_all[metoda]

    meta_model = RandomForestRegressor(100)
    res_meta = cross_validate(meta_model, x_meta, y_meta, scoring="neg_mean_squared_error")
    rmse_meta = np.sqrt(-res_meta['test_score'].mean())

    dummy_model = DummyRegressor()
    res_dummy = cross_validate(dummy_model, x_meta, y_meta, scoring="neg_mean_squared_error")
    rmse_dummy = np.sqrt(-res_dummy['test_score']).mean()

    print(f"RMSE meta modela za {metoda}: {rmse_meta}")
    print(f"RMSE dummy modela za {metoda}: {rmse_dummy}\n")

A.DODATNO: S posamičnimi regresijskimi meta modeli za natančost iz A.3 lahko napovedujemo tudi najboljši model, tako da pogledamo, katerera od napovedanih natančnosti je najvišja. Primerjaj napovedi najboljšega modela z meta klasifikacijo in z meta regresijo. Katera se obnese bolje?

### B: Optimizacija hiperparametrov
Vse razen najpreprostejših metod strojnega učenja imajo eno ali več nastavitev, ki jim pravimo hiperparametri. Primerna nastavitev hiperparametrov za dane podatke je pogosto prav tako pomembna kot izbira primerne metode. V knjižnici scikit-learn lahko seznam hiperparametrov vsake metode najdemo s klicem **model.get_params()**, da jih zares razumemo, pa je treba pogledati v dokumentacijo ali pa celo prebrati članek o metodi. 

B.1 Naredi model odločitvenega drevesa ter se poigraj z njegovimi hiperparametri. Verjetno bosta najpomembnejša **max_depth** in **min_samples_split**. Za računje uspešnosti modela uporabi metriko $R^2$ in prečno preverjanje.

In [48]:
import numpy as np
import pandas as pd

data = pd.read_csv("drugi_del_podatki.csv")
x = data.drop("y", axis=1)
y = data["y"]
print(y)

0      0.763052
1      0.417707
2      0.167011
3      1.034676
4      0.512737
         ...   
995    0.608756
996    0.707602
997    0.217132
998    1.055196
999    1.170648
Name: y, Length: 1000, dtype: float64


In [65]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


tre = DecisionTreeRegressor(max_depth=100, min_samples_split=5)
cvtre = cross_validate(tre, X_train, y_train, cv=5,
    scoring=(['r2']),
    return_train_score=True)

r2 = cvtre['test_r2'].mean()
print(r2)


0.20649708104983802


B.2 Iskanje najboljših hiperparametrov lahko razumemo kot optimizacijski problem. Sci-kit learn nam ponuja nekaj orodij za avtomatsko optimizacijo. Najpreprostejša je **sklearn.model_selection.GridSearchCV**, ki preišče celoten kartezični produkt vrednosti hiperparametrov, ki nas zanimajo.
Uporabi grid search, da preiščeš kombinacije hiperparametrov **2 <= max_depth < 50** in **2 <= min_samples_split < 400**. 
Metodi grid search nastavi **cv=5**, da bo izvajala notranje prečno preverjanje. Koristna nastavitev je tudi **refit=True**.

Pomembno se je zavedati, da pri resni optimizaciji hiperparametrov prilagajamo model na celotno množico podatkov. Zato je dobra praksa pred optimizacijo ločiti podatke na učno in testno množico ter izvesti optimizacijo na učni množici, končni model pa evaluirati na testni množici. 
(Obstaja celo možnost gnezdenega prečnega preverjanja, a to pustimo za drugič.)

Po opravljeni optimizaciji hiperparametrov lahko dobimo najboljše vrednosti z **grid_search.best_params_**, natreniran najboljši model pa z **grid_search.best_estimator_** (če smo nastavili refit=True).

In [63]:
from sklearn.model_selection import GridSearchCV

np.random.seed(0)

tree_param = [{'max_depth': range(2,50,3), 'min_samples_split': range(2,400, 7)}]
grid_search = GridSearchCV(DecisionTreeRegressor(), tree_param, cv=5, refit=True)
grid_search.fit(X_train, y_train)



print(grid_search)
print(grid_search.best_params_)
print(grid_search.best_estimator_)



# DOPOLNI

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'max_depth': range(2, 50, 3),
                          'min_samples_split': range(2, 400, 7)}])
{'max_depth': 8, 'min_samples_split': 51}
DecisionTreeRegressor(max_depth=8, min_samples_split=51)


In [64]:
from sklearn.metrics import r2_score

bestTree = grid_search.best_estimator_

pred = bestTree.predict(X_test)
R2 = r2_score(y_test, pred)
print('R2:',R2)

R2: 0.36163014968385987


B.3 Nariši še 2D vizualizacijo metrike $R^2$ za vse preizkušene kombinacije hiperparametrov. Pomagaš si lahko z **grid_search.param_grid**, **np.reshape**, **plt.imshow**.

In [ ]:
import matplotlib.pyplot as plt

# glej resitve

B.DODATNO: poglej si in preizkusi še **RandomizedSearchCV**, **HalvingGridSearchCV** ter **HalvingRandomSearchCV**. Opisani so tukaj: https://scikit-learn.org/stable/modules/grid_search.html.

RandomizedSearchCV...nakljucno izbira parametre iz poradelitve moznih parametrov